Hello All, This is my very first notebook. I'm a novice in Machine Learning therefore, when you will go through this notebook, you may not find anything interesting. But,I'm very excited to share my understanding on FakeNewsDetction Dataset. I would really appreciate your suggestions. Happy Learning !!

Little description of this notebook: This is mainly focusing on Natural Language Processing techniques such as removal of stopwords and stemming. Suggestions to innovate this would be really appresciated. :)

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# NLP libraries to clean the text data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

# Vectorization technique TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# For Splitting the dataset
from sklearn.model_selection import train_test_split

# Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#Accuracy measuring library
from sklearn.metrics import accuracy_score


## 2. Loading the data

In [ ]:
file_path = '../input/fake-news-detection/data.csv'
data = pd.read_csv(file_path)

In [ ]:
data.shape #Returns the number of rows and columns present in the dataset

In [ ]:
data.head()  # Returns the first 5 rows of the dataset

In [ ]:
data.columns # Returns the column headings

In [ ]:
data.isnull().sum() #To check the null values in the dataset, if any

## 3.Data-Preprocessing

For further analysis, cleaning of data is necessary. 
In this Notebook, I will be doing 3 stages of data cleaning:
1. Removing the Null values
2. Adding a new field
3. Drop features that are not needed
4. Text Processing

In [ ]:
df = data.copy() #Creating a copy of my data, I will be working on this Dataframe

## 3.1. Removing the Null Values

As Body field has some empty fields, it can be handled in two ways:
1. Drop the 21 rows
2. Replace the null value with a dummy string

Here, I will be going with the 2nd option, because although dropping 21 rows would not affect the accuracy, as it is just a minute portion of our large dataset, it is never recommended.

I will be replacing the Null(Nan) values in 'Body' field with an empty string ('')

In [ ]:
df['Body'] = df['Body'].fillna('')   # As Body is empty, just filled with an empty space

In [ ]:
df.isnull().sum()  # No null values found

## 3.2. Adding a new column
For ease of implementation, I combined Headline and Body Column and created a new column 'News' 

In [ ]:
df['News'] = df['Headline']+df['Body']

In [ ]:
df.head()

In [ ]:
df.columns

## 3.3. Drop features that are not needed

In [ ]:
features_dropped = ['URLs','Headline','Body']
df = df.drop(features_dropped, axis =1)

In [ ]:
df.columns

## 3.4. Text Processing
1. Remove symbols(',','-',...etc)
2. Remove stop words
3. Stemming

In [ ]:
ps = PorterStemmer()
def wordopt(text):
    text = re.sub('[^a-zA-Z]', ' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [ ]:
df['News'] = df['News'].apply(wordopt) #Applying the text processing techniques onto every row data

In [ ]:
df.head()

## Splitting DataSet

In [ ]:
X = df['News']
Y = df['Label']

#Split the data into training and test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

## 5. Vectorization

This is used to handle our text data, by converting it into vectors.

In [ ]:
#Vectorization
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## 6. Model Fitting
I will be fitting my data onto 3 classifications models
1. Logistic Regression
2. SVM
3. RandomForestClassifier

The best one amongst the 3 will be used further

In [ ]:
#1. Logistic Regression - used because this model is best suited for binary classification
LR_model = LogisticRegression()

#Fitting training set to the model
LR_model.fit(xv_train,y_train)

#Predicting the test set results based on the model
lr_y_pred = LR_model.predict(xv_test)

#Calculate the accurracy of this model
score = accuracy_score(y_test,lr_y_pred)
print('Accuracy of LR model is ', score)

In [ ]:
#2. Support Vector Machine(SVM) - SVM works relatively well when there is a clear margin of separation between classes.
svm_model = SVC(kernel='linear')

#Fitting training set to the model
svm_model.fit(xv_train,y_train)

#Predicting the test set results based on the model
svm_y_pred = svm_model.predict(xv_test)

#Calculate the accuracy score of this model
score = accuracy_score(y_test,svm_y_pred)
print('Accuracy of SVM model is ', score)

In [ ]:
#3. Random Forest Classifier 
RFC_model = RandomForestClassifier(random_state=0)

#Fitting training set to the model
RFC_model.fit(xv_train, y_train)

#Predicting the test set results based on the model
rfc_y_pred = RFC_model.predict(xv_test)

#Calculate the accuracy score of this model
score = accuracy_score(y_test,rfc_y_pred)
print('Accuracy of RFC model is ', score)

## 7. Manual Model Testing

In [ ]:
# As SVM is able to provide best results - SVM will be used to check the news liability

def fake_news_det(news):
    input_data = {"text":[news]}
    new_def_test = pd.DataFrame(input_data)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    #print(new_x_test)
    vectorized_input_data = vectorization.transform(new_x_test)
    prediction = svm_model.predict(vectorized_input_data)
    
    if prediction == 1:
        print("Not a Fake News")
    else:
        print("Fake News")

In [ ]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

In [ ]:
fake_news_det("""The second Covid-19 wave in India is now on the "downswing," the Centre said on Thursday, highlighting that the current number of active cases is still "very high" and advised states and Union territories (UTs) to not let down their guards.""")

In [ ]:
fake_news_det("JetNation FanDuel League; Week 4 of readers think this story is Fact. Add your two cents.(Before Its News)Our FanDuel league is back again this week. Here are the details:$900 in total prize money. $250 to the winner. $10 to enter.Remember this is a one week league, pick your lineup against the salary cap and next week if you want to play again you can pick a completely different lineup if you want.Click this link to enter — http://fanduel.com/JetNation You can discuss this with other NY Jets fans on the Jet Nation message board. Or visit of on Facebook.Source: http://www.jetnation.com/2017/09/27/jetnation-fanduel-league-week-4/")